In [1]:
import os
import pandas as pd

/var/folders/mx/d86l02y96_744bc55lr4kb880000gn/T/ipykernel_75207/3571106454.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Import data

In [2]:
base_dir = '/Users/whybless/Documents/ai/nlp/sentiment_analysis'
data_dir = os.path.join(base_dir, 'stock_data.csv')
df = pd.read_csv(data_dir)
df.head()

,Text,Sentiment
0,Kickers on my watchlist XIDE TIT SOQ PNK CPW B...,1
1,user: AAP MOVIE. 55% return for the FEA/GEED i...,1
2,user I'd be afraid to short AMZN - they are lo...,1
3,MNTA Over 12.00,1
4,OI Over 21.37,1


In [3]:
from sklearn.feature_extraction.text import re

def clean_data(review):
    
    no_punc = re.sub(r'[^\w\s]', '', review)
    no_digits = ''.join([i for i in no_punc if not i.isdigit()])
    
    return(no_digits)
df['Text'] = df['Text'].apply(clean_data)



### train-test split:
from sklearn.model_selection import train_test_split
import nlpaug.augmenter.word as naw

def augment_text(text):
    aug = naw.ContextualWordEmbsAug(model_path='bert-base-uncased', action="insert")
    return aug.augment(text)

y = df['Sentiment'] # target variable
df['Text'] = df['Text'].apply(augment_text)
X_train, X_test, y_train, y_test = train_test_split(X,y)


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer


tfidf = TfidfVectorizer(strip_accents=None, 
                        lowercase=False,
                        preprocessor=None)

X = tfidf.fit_transform(df['Text'])

In [5]:
from sklearn.model_selection import train_test_split
y = df['Sentiment'] # target variable
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [6]:
import numpy as np
unique_classes = np.unique(y_train)
print(unique_classes)

[-1  1]


In [7]:
#logistic regression classifier on the training dataset and use it to make predictions on the test data
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver='liblinear')
lr.fit(X_train,y_train) # fit the model
preds = lr.predict(X_test) # make predictions

In [8]:
from sklearn.metrics import accuracy_score
accuracy_score(preds,y_test)  #76 before aug

0.7734806629834254

In [9]:
# Save 
import joblib 
joblib.dump(lr, 'logistic_regression_model.pkl')

['logistic_regression_model.pkl']